# Camera robot kalibratie

> Uit vorige opdrachten hebben we gezien hoe we pixelcoördinaten kunnen omzetten naar metrische coördinaten in cameraframe {C} via de intrinsieke matrix, en verder kunnen omzetten naar metrische coördinaten in een arbitrair frame {T}, bepaald door een dambordpatroon, via de extrinsieke matrix.

> Dit is exact wat we nodig hebben voor onze robot manipulatie. Dit met het verschil dat we de punten niet nodig hebben in een arbitrair target frame {T} maar in het robot baselinkframe {B}. We hebben dus in totaal een matrix ${}^{B}T_{C}$ nodig die punten omzet van cameraframe {C} naar robot baseframe {B}. We kunnen hier gebruik maken van een handige techniek genaamd 'eye-to-hand' kalibratie. Dit wil zeggen dat we ons dambordpatroon zullen bevestigen aan de robot end-effector en hier beelden van zullen nemen met de camera. Op deze manier kunnen we terug door extrinsieke kalibratie de transformatiematrix ${}^{C}T_{T}$ van het cameraframe {C} naar het targetframe {T} bepalen, waarbij dit targetframe is bevestigd aan de robot end-effector. Als we matrix ${}^{B}T_{C}$ willen bekomen via matrix ${}^{C}T_{T}$, dan ontbreekt er nog de matrix ${}^{B}T_{T}$. Dit is de transformatie van de robot baselink naar het target frame. We krijgen dan:

$${}^{B}T_{C}={}^{B}T_{T}*{}^{T}T_{C}={}^{B}T_{T}*({}^{C}T_{T})^{-1}$$

> De totale transformatie van het robot baselinkframe naar het targetframe ${}^{B}T_{T}$ betsaat uit twee delen. Enerzijds bestaat dit uit de transformatie ${}^{B}T_{ee}$ van het robot baseframe naar het robot end-effector frame. Dit kennen we uit de voorwaartse kinematica van de robot die berekend wordt uit de joint waardes. Anderzijds bestaat deze uit de transformatie van de robot end-effector tot het dambordpatroon (frame {T}). Deze is gekend als we de exacte positie kennen van het dambordpatroon ten opzichte van de end-effector ${}^{T}T_{ee}$. Deze kennen we als we het dambordpatroon monteren in een gekende positie op de end-effector. Aangezien we beide kennen hebben we:

$${}^{B}T_{T}={}^{B}T_{ee}*{}^{ee}T_{T}$$

> Als we dus naast deze twee gekende matrices de extrinsieke kalibratiematrix kunnen bepalen, dan krijgen we de totale matrix ${}^{B}T_{C}$ die punten gedefiniëerd in het camera frame kan projecteren in het baselink frame. En eens de robot de positie van het object kent in baselink frame, dan kan deze daar naartoe bewegen. De totale matrix vermenigvuldiging is:

$${}^{B}T_{C}={}^{B}T_{ee}*{}^{ee}T_{T}*{}^{T}T_{C}={}^{B}T_{ee}*{}^{ee}T_{T}*({}^{C}T_{T})^{-1}$$

# 1. Importeer bibliotheken

> Deze bibliotheken zijn nodig voor het volbrengen van dit script. In de CameraCalibration bibliotheek (die je vindt in de Classes folder op je Google Drive) vind je de code achter de commando's die je hier zal gebruiken. Deze cel zal een error geven als je niet met je Google Drive bent verbonden. Zorg er dus voor dat je links in de bestanden een folder 'drive' ziet staan naast de default 'sample_data'-folder. Indien niet, verbind met de drive door links op het mapje te klikken en vervolgens op het Google Drive symbool te klikken. Druk vervolgens nog eens op 'Runtime', 'Runtime opnieuw starten', om te refreshen.



In [ ]:
import sys
sys.path.append('/content/drive/My Drive/object_pose_estimation_online')
from Classes.CameraCalibration import *
from google.colab.patches import cv2_imshow

# 2. Importeer alle beelden voor camera-robot kalibratie

> Alle afbeeldingen bevinden zich onder de map /content/drive/My Drive/object_pose_estimation_online/data/camera_robot_calibration_images. Als je dit uitprint kan je zien dat dit een array is met alle 10 de padnamen naar elke afbeelding.

In [ ]:
# Get all image's path names
robot_camera_calibration_images_file = '/content/drive/My Drive/object_pose_estimation_online/data/camera_robot_calibration_images/CRC_image_original_*.jpg'
images_path_names = get_image_path_names(robot_camera_calibration_images_file)
print(images_path_names)



> We kunnen terug kijken hoe een dergelijke afbeelding eruit ziet. Merk op dat het dambordpatroon deze keer wordt vastgehouden door de robot.



In [ ]:
# Preview image
image_path = images_path_names[0]
image = read_image(image_path)
cv2_imshow(image)

# 3. Laden van de baselink-target transformatie matrix

> Voor een gegeven afbeelding van het dambordpatroon zullen we de transformatie matrix ${}^{C}T_{T}$ berekenen van camera frame naar target frame door middel van extrinsieke kalibratie. Om de gewenste baselink-camera transformatie ${}^{B}T_{C}$ te bekomen, hebben we nog de transformatiematrix nodig van baselink naar het target ${}^{B}T_{T}$, dit komt uit onze voorwaartse kinematica van de robot en hebben we reeds opgeslagen in /content/drive/My Drive/object_pose_estimation_online/data/camera_robot_calibration_images/baselink_target_transformations.npy. Je kan deze array printen en zien dat deze 10 keer een 4x4 transformatiematrix bevat (we hebben data van 10 afbeeldingen opgenomen). Per afbeelding hebben we dus de totale ${}^{B}T_{T}$ matrix opgeslagen.



In [ ]:
# Get all corresponding baselink-target transformation matrices (numpy data)
baselink_target_transformations_file = '/content/drive/My Drive/object_pose_estimation_online/data/camera_robot_calibration_images/baselink_target_transformations.npy'
baselink_target_transformations = get_numpy_data(baselink_target_transformations_file)
print(np.shape(baselink_target_transformations))



> We hebben hierboven de eerste afbeelding (index 0) ingeladen en hebben nu dus ook de eerste baselink-target transformatie nodig uit de array.



In [ ]:
# Get corresponding Baselink - Target transform for the 0th image
bt_transform = baselink_target_transformations[0]
print("Baselink-target transform: \n\n" + str(bt_transform))

> Deze matrix stelt dus de transformatiematrix voor van baselink frame {B} naar het dambordframe of targetframe {T}, ${}^{B}T_{T}$. Of in andere woorden, deze matrix kan punten beschreven in het target frame omzetten in het baselink frame.

$$ {}^{B}P = {}^{B}T_{T}*{}^{T}P$$


# 4. Laden van de berekende intrinsieke camera matrix

> In vorige opgave heb je de intrinsieke matrix opgelsagen, deze werd automatisch opgeslagen onder /content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz. Deze kunnen we nu laden. De functie "load_intrinsic_camera_matrix" geeft je niet enkel de intrinsieke matrix, maar ook de distrotiecoëfficiënten nodig voor de kalibratie.





In [ ]:
# Load intrinsic camera matrix and distortion coefficients
intrinsic_camera_matrix_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz'
mtx, dist = load_intrinsic_camera_matrix(intrinsic_camera_matrix_file)

# 5. Camera-robot kalibratie

> In dit deel wordt de camera-robot kalibratie uitgevoerd. Dit door terug een extrinsieke kalibratie uit te voeren en deze matrix te inverteren en te vermenigvuldingen met de baselink-target matrix. Je bekomt dus de baselink-camera transformatiematrix ${}^{B}T_{C}$.


In [ ]:
# Prepare object points in 3D space in meters
objp = get_object_points()

# Read image
image = read_image(image_path)

# Turn image to grayscale
gray = image_to_grayscale(image)

# Find the corners in the chessbord calibration tool
corners = find_corners(gray)

# Get extrinsic camera calibration matrix (camera-target transform)
ct_transform = extrinsic_calibration(objp, corners, mtx, dist)
print("Camera-target transform: \n\n" + str(ct_transform))



> Nu dat je de transformatiematrix van het cameraframe naar het targetframe ${}^{C}T_{T}$ hebt, kan de deze gebruiken samen met de ${}^{B}T_{T}$ matrix om de totale ${}^{B}T_{C}$ matrix te bekomen. Let op dat je ${}^{C}T_{T}$ eerst nog moet inverteren! De baselink-camera matrix bereken je namelijk als volgt:

$${}^{B}T_{C} = {}^{B}T_{T} * {}^{T}T_{C} $$



In [ ]:
# Invert the transform
tc_transform = invert_transform(ct_transform)

# Compute Baselink - Camera transform using the "multiply_transforms" function
bc_transform = multiply_transforms(bt_transform, tc_transform)

# Save the mean baselink - camera transformation matrix
baselink_camera_transformation_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/baselink_camera_transformation'
save_to_numpy(baselink_camera_transformation_file, bc_transform)

print("Baselink-camera transform: \n\n" + str(bc_transform))

# 6. Test robot-camera calibration

> In wat volgt kunnen we de robot-camera kalibratie testen. De inverse van de berekende ${}^{B}T_{C}$ kan namelijk punten gedefiniëerd in robot frame omzetten in camera frame. Samen met de intrinsieke matrix kunnen we dus punten omzetten van robot frame in pixel frame en dus virtueel geplaatste objecten in robot frame visualiseren in de afbeelding. We definiëren hier een kubus in coördinaten gedefiniëerd ten opzichte van het robot frame.






In [ ]:
# Define axis and box coordinates in world coordinate system
box = np.float32([[-0.1, -0.6, 0], [-0.1, -0.7, 0], [-0.2, -0.7, 0],
                  [-0.2, -0.6, 0], [-0.1, -0.6, -0.1], [-0.1, -0.7, -0.1], [-0.2, -0.7, -0.1],
                  [-0.2, -0.6, -0.1]]).reshape(-1, 3)



> We willen nu punten gedefiniëerd in baselinkframe omzetten in camera frame. Dus hebben we de geïnverteerde nodig van ${}^{B}T_{C}$, namelijk ${}^{C}T_{B}$.



In [ ]:
# Invert transform
cb_transform = invert_transform(bc_transform)


> Uit de berekende baselink-camera matrix kunnen we afzonderlijk het rotatie gedeelte en het translatiegedeelte uithalen. Deze dienen we afzonderlijk aan het kalibratie algoritme te geven.

$${}^{C}T_{B} = \begin{bmatrix} r_{11} & r_{12} & r_{13} & t_{x} \\ r_{21} & r_{22} & r_{23} & t_{y} \\ r_{31} & r_{32} & r_{33} & t_{z}\\ 0 & 0 & 0 & 1 \end{bmatrix} ; {}^{C}R_{B} = \begin{bmatrix} r_{11} & r_{12} & r_{13}\\ r_{21} & r_{22} & r_{23}\\ r_{31} & r_{32} & r_{33}\end{bmatrix} ; {}^{C}t_{B} = \begin{bmatrix} t_{x} \\ t_{y} \\ t_{z}\end{bmatrix}$$

In [ ]:
# Retrieve rotation matrix and translation vector from cb_transform
cb_rot = cb_transform[:3, :3]
cb_trans = cb_transform[0:3, 3]

> Hier projecteren we de gedefiniëerde metrische coördinaten van de kubus (box) beschreven in het baselink frame in pixelcoördinaten en tekenen we deze op de afbeelding. Merk terug op dat aan de functie 'cv2.projectPoints' nu de translatie en rotatie matrices van het cameraframe naar het baselink frame worden meegegeven, dit was niet zo in het vorige script, daar waren dit de matrices van camera frame naar target frame.

In [ ]:
# Project 3D box points to pixel coordinates
imgpts_box, _ = cv2.projectPoints(box, cb_rot, cb_trans, mtx, dist)

# Draw box on image
image = draw_box1(image, imgpts_box.astype(int))



> Plot hieronder de afbeeldingen met de geprojecteerde pixels.






In [ ]:
# Show result
cv2_imshow(image)


> Het resultaat laat inderdaad zien dat de gedefiniëerde kubus in baselink frame coördinaten mooi de juiste pixels triggert om het effect te creëren dat deze effectief in de ruimte aanwezig is. Je brein zal hoogst waarschijnlijk even de tijd nodig hebben om te zien dat de kubus wel degenlijk correct geplaatst is voor de robot. Je kan zelf wat spelen door de coördinaten van de kubus te veranderen.



> Je kan ook zelf kijken welke pixels worden getriggerd door punten in metrische coördinaten, gedefiniëerd in het baselink frame te projecteren in pixel frame.

In [ ]:
# Define point
point = np.float32([0, 0, 0])

# Project 3D point to pixel coordinates
imgpts_point, _ = cv2.projectPoints(point, cb_rot, cb_trans, mtx, dist)
print(imgpts_point)

# 7. Omzetten van het coördinaat in camera frame naar het coördinaat in robot frame

Door de uitgevoerde calibratie hebben we nu een matrix die metrische coördinaten in camera frame kan omzetten naar metrische coördinaten in robot frame. We kunnen dus nu het coördinaat van het object in camera frame uit vorig script omzetten naar coördinaten in robot frame. Dit doen we met de hierboven bepaalde baselink-camera transformatie.

In [ ]:
# Load camera coordinate
obj_camera_coor = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/obj_camera_coor.npy'
camera_coordinate = get_numpy_data(obj_camera_coor)
print(camera_coordinate)

In [ ]:
# Covert position to baselink frame
b_p = np.dot(bc_transform,  camera_coordinate)
print("Coördinaat in baselink frame: \n\n" + str(np.reshape(b_p, (4, 1))))

Dit is dus het centerpunt van het object gedefiniëerd in het baselink frame van de robot. Om de robot naar een positie te kunnen sturen hebben we niet enkel een positie nodig maar hebben we een volledige transformatiematrix nodig van baselink naar target. Hieronder definiëren we nog welke oriëntatie we bij het object willen hebben door custom een 3x3 rotatiematrix te definiëren. Vervolgens brengen we die custom rotatiematrix en de berekende translatie matrix (positie) samen tot een volwaardige transformatiematrix.

In [ ]:
# Create target orientation (custom)
R_target = np.array([[math.sqrt(2)/2, -math.sqrt(2)/2, 0],
       [-math.sqrt(2)/2, -math.sqrt(2)/2, 0],
       [0, 0, -1]])
print("Rotatie matrix: \n\n" + str(R_target))

# Create target position (calculated from image)
P_target = b_p
print("\nTranslatie matrix: \n\n" + str(np.reshape(P_target, (4, 1))))

# Create total transformation (R and T)
bt_transform = np.zeros((4, 4))
bt_transform[:3, :3] = R_target
bt_transform[:, 3] = P_target
print("\nTotale transformatiematrix: \n\n" + str(bt_transform))

Dit is wat we wilden bekomen. We kunnen nu als aller laatste stap deze matrix opslaan en deze in het volgende script visualiseren en de robot ernaar toe laten bewegen.

In [ ]:
# Save the baselink - target transformation matrix
baselink_target_transformation_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/baselink_target_transformation'
save_to_numpy(baselink_target_transformation_file, bt_transform)

# Vragen bij dit script:

*  VRAAG 1: Wat is de x-, y-, en z- positie van de oorsprong van het targetframe {T} in het baselink frame {B} op het moment van het nemen van de eerste foto?
*  VRAAG 2: Wat is de x-, y-, en z- positie van de oorsprong van het targetframe {T} in het camera frame {C} op het moment van het nemen van de eerste foto?
*  VRAAG 3: Welk pixelcoördinaat wordt er getriggerd als een punt op coördinaat (0, -0.6, 0.0) meter in het baselink frame wordt geplaatst?